# Experimental pipeline for GEQIE

## Setup

### Imports

In [39]:
import json
import subprocess
import tempfile

import pandas as pd

from concurrent import futures
from pathlib import Path
from typing import Any, Dict

In [2]:
%load_ext ipyform
%form_config --auto-detect 1

### Configurations

In [62]:
# @markdown #### Input and output paths
input_path = "../assets/grayscale" # @param {type:"string"}
output_path = "../.outputs"        # @param {type:"string"}


# @markdown #### Encoding 
encoding_method = "frqi"           # @param {type:"string"}
grayscale = True                   # @param {type:"boolean"}
n_shots = 1024                     # @param {type:"integer"}
return_padded_counts = True        # @param {type:"boolean"}

# @markdown #### Concurrency config
max_concurrency = None             # @param {type:"integer"}
# @markdown **NOTE:** Type `None` for default value of `max_concurrency` (number of all available system CPUs)

input_path = Path(input_path)

FormWidget(children=(VBox(children=(HTML(value=''), HTML(value='<h4>Input and output paths</h4>'), Box(childre…

### Processing functions

In [63]:
# @title Processing Functions

def process_file(
        input_path: str, 
        encoding_method: str, 
        n_shots: int, 
        grayscale: bool,
        return_padded_counts: bool,
        workdir: str,
) -> Dict[str, Any]:
    process_result = subprocess.run(
        f"geqie simulate \
            --encoding {encoding_method} \
            --image {input_path} \
            --n-shots {n_shots} \
            --return-padded-counts {return_padded_counts} \
            --grayscale {grayscale}",
        capture_output=True,
    )

    if process_result.returncode:
        raise RuntimeError(process_result.stderr.decode("utf-8"))

    with tempfile.TemporaryFile(dir=workdir, mode="w", delete=False) as output_file:
        df = pd.DataFrame(
            columns=["file_name", "result"],
            data=[[
                input_path,
                json.dumps(json.loads(process_result.stdout.decode("utf-8")))
            ]]
        )
        df.to_csv(output_file, sep="\t", index=False)

    return df

def process_directory(
        input_path: Path, 
        encoding_method: str, 
        n_shots: int, 
        grayscale: bool,
        return_padded_counts: bool, 
        workdir: str,
) -> Dict[str, Any]:
    workdir_path = Path(workdir)
    input_files = input_path.glob("*")

    with futures.ThreadPoolExecutor() as executor:
        execute_futures = [
            executor.submit(
                process_file,
                input_file_path,
                encoding_method,
                n_shots,
                grayscale,
                return_padded_counts,
                workdir,
            ) for input_file_path in input_files
        ]

    for f in execute_futures:
        f.result()

    dfs = []
    for temp_file_path in workdir_path.glob("*"):
        dfs.append(pd.read_csv(temp_file_path, sep="\t"))
    
    return pd.concat(dfs)


### Run

In [64]:
# @title Run

def run():
    with tempfile.TemporaryDirectory() as workdir:
        if input_path.is_file():
            df = process_file(
                input_path=input_path,
                encoding_method=encoding_method,
                n_shots=n_shots,
                return_padded_counts=return_padded_counts,
                grayscale=grayscale,
                workdir=workdir,
            )
        if input_path.is_dir():
            df = process_directory(
                input_path=input_path,
                encoding_method=encoding_method,
                n_shots=n_shots,
                return_padded_counts=return_padded_counts,
                grayscale=grayscale,
                workdir=workdir,
            )

    print(df)

run()

                                 file_name  \
0  ..\assets\grayscale\test_image_1010.png   
0         ..\assets\grayscale\test_2x4.png   
0   ..\assets\grayscale\test_image0xf0.png   
0       ..\assets\grayscale\test_image.png   
0       ..\assets\grayscale\full_white.png   
0   ..\assets\grayscale\test_image_4x4.png   
0    ..\assets\grayscale\test_flag_4x4.png   

                                              result  
0  {"000": 0, "001": 237, "010": 96, "011": 174, ...  
0  {"00000": 0, "00001": 65, "00010": 0, "00011":...  
0  {"000": 132, "001": 106, "010": 138, "011": 10...  
0  {"000": 0, "001": 259, "010": 37, "011": 231, ...  
0  {"000": 0, "001": 251, "010": 0, "011": 253, "...  
0  {"00000": 0, "00001": 72, "00010": 58, "00011"...  
0  {"00000": 0, "00001": 59, "00010": 0, "00011":...  
